In [2]:
# ! pip install langchain
# ! pip install openai

In [3]:
# ! pip install unstructured
# ! pip install libmagic
# ! pip install langchain

In [4]:
from apikey import OPEN_AI_KEY
from langchain.llms import OpenAI

In [5]:
import os 
# add your openai api key
os.environ['OPENAI_API_KEY'] = " "

In [6]:
from langchain.document_loaders import UnstructuredFileLoader

# load documents you want to llm to answer from.
loaders = UnstructuredFileLoader(r"C:\Users\hrush\Downloads\About Us.docx")
data = loaders.load()

In [7]:
data

[Document(page_content="About Us: \n\nAdVantageAI- Automate and Amplify your business, Digitally\n\nAt AdVantageAI, we are a dynamic agency that combines the power of AI automation and social media marketing to drive success for businesses. We understand the importance of both cutting-edge technologies and effective digital strategies in today's competitive landscape. Our mission is to empower businesses to thrive by harnessing the potential of AI automation and leveraging the reach and influence of social media platforms.\n\nServices We Offer:\n\n1. AI Automation Services:\n\n   - Conversational AI Development: Utilize NLP and Amazon Lex to build chatbots, virtual assistants, and voice-based interfaces that can interact with users, understand their intent, and provide relevant responses or perform automated tasks. (Problem Statement: Enhance customer experience and reduce response times by automating customer interactions and queries.)\n\n   - Customer Support Automation: Develop AI-p

In [8]:
from langchain.text_splitter import CharacterTextSplitter

# This splits the data into chunks 
text_splitter = CharacterTextSplitter(separator = '\n',
                                      chunk_size=1000,
                                      chunk_overlap=200)

docs = text_splitter.split_documents(data)

In [9]:
len(docs)

11

In [10]:
import pickle
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [11]:
persist_directory = 'db'

# Create a vector database using chroma
vectordb = Chroma.from_documents(
    docs,
    embeddings,
    persist_directory=persist_directory)


In [12]:
# persist the database 
vectordb.persist()
vectordb = None

In [13]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)

In [14]:
# retriever to perform semantic search
retriever = vectordb.as_retriever()

In [16]:
from langchain.chains import RetrievalQA

# create a chain to retrieve the info and print the answer for the query
chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                    chain_type='stuff',
                                    retriever=retriever,
                                    return_source_documents=True)


In [18]:
def process_llm_response(llm_response):
    print(llm_response['result'])

In [20]:
query = 'Services offered?'
llm_response = chain(query)
print(process_llm_response(llm_response))

 AdVantage Digital offers service fee only and complete package options for their clients. Under the service fee only option, clients pay a service fee for the selected marketing services and are responsible for covering the costs of email marketing and ad running fees. Under the complete package option, clients can choose a one-time charge for the services, ads, and email marketing, allowing for a more seamless management of their marketing needs.
None


In [22]:
query1 = "What are the AI Automation services?"
llm_response1 = chain(query1)
print(process_llm_response(llm_response1))

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-CKPy6n4fS0ihzAAnvLHke5nG on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


 AdVantage Digital offers two different contract options. The first option is a service fee only, where clients pay a service fee for the selected marketing services and are responsible for covering the costs of email marketing and ad running fees. The second option is a complete package, where AdVantage Digital provides a one-time charge for the services, ads, and email marketing. This comprehensive option allows for seamless management of marketing needs without the hassle of separate billing.
None
